# ATLAS Integration Challenge

The next cell updates dependencies, make sure to restart the kernel afterwards.

In [ ]:
! pip install --upgrade --quiet atlas_schema
! pip install --upgrade --quiet --pre mplhep

In [ ]:
import collections
import gzip
import json
import math
import pathlib
import time

import awkward as ak
import cloudpickle
import dask
import dask.bag
import hist
import matplotlib.pyplot as plt
import mplhep
import numpy as np
import uproot
import vector

from atlas_schema.schema import NtupleSchema
from coffea import processor
from coffea.nanoevents import NanoEventsFactory
from dask.distributed import Client, PipInstall, performance_report
from IPython.display import display

import utils

vector.register_awkward()
mplhep.style.use(mplhep.style.ATLAS1)

client = Client("tls://localhost:8786")

plugin = PipInstall(packages=["atlas_schema"], pip_options=["--upgrade"])
client.register_plugin(plugin)

cloudpickle.register_pickle_by_value(utils)

# utils.new_output_path()  # optional: create new path to store output in

### Fileset preparation

Comment out the desired version in the next cell to run over the full input dataset or a subset.

In [ ]:
# full scale
# fileset, input_size_GB = utils.get_fileset(campaign_filter=None, dsid_filter=None, max_files_per_sample=None)

# 9% of full scale, using 2022 data + MC
# fileset, input_size_GB = utils.get_fileset(campaign_filter=["mc23a", "data22"], dsid_filter=None, max_files_per_sample=None)

fileset, input_size_GB = utils.get_fileset(campaign_filter=["mc23a", "mc23d", "data22", "data23"], dsid_filter=None, max_files_per_sample=None)

# minimal setup for debugging
# fileset, input_size_GB = utils.get_fileset(campaign_filter=["mc23a"], dsid_filter=["601229"], max_files_per_sample=10)

### 200 Gbps-style `uproot.iterate` version

Alternative setup in the style of https://github.com/iris-hep/idap-200gbps-atlas/.

In [ ]:
# import tqdm.notebook

# all_files = []
# for k, v in fileset.items():
#     all_files += v["files"] if "period" not in k else []  # skip data (which has less branches)

# columns_to_read = json.load(pathlib.Path("columns_to_preload.json").open())["all_systematics"]

# def read_branches(fname, branches):
#     try:
#         t0 = time.time()
#         with uproot.open(fname) as f:
#             for _ in f["reco"].iterate(expressions=branches, step_size=25_000):
#                 pass
#             size_read = f.file.source.num_requested_bytes / 1_000**2
#         t1 = time.time()
#         return {"t0": t0, "t1": t1, "MBread": size_read}
#     except:
#         return {"t0": 0, "t1": 0, "MBread": 0, "fname": fname}

# t0 = time.time()
# tasks = [dask.delayed(read_branches)(fname, columns_to_read) for fname in all_files]
# futures = client.compute(tasks)

# with tqdm.notebook.tqdm(total=len(futures)) as pbar:
#   for future in dask.distributed.as_completed(futures):
#     pbar.update(1)

# res = [f.result() for f in futures]
# t1 = time.time()

# GBread_total = sum([r["MBread"] for r in res])/1_000
# print(f"{GBread_total:.2f} GB read total in {t1-t0:.1f} sec, {GBread_total * 8 / (t1-t0):.2f} Gbps, fraction read: {GBread_total/input_size_GB:.2%}")

# # debug failures
# failed_files = [r.get("fname") for r in res if r.get("fname") is not None]
# print(f"{len(failed_files)} failures: {failed_files}")

# for failed in failed_files:
#     read_branches(failed, columns_to_read)

### Simple non-distributed reading

This cell is for debugging and to ensure the basic non-distributed infrastructure works.

In [ ]:
class TCPTSchema(NtupleSchema):
    # mcChannelNumber not defined for data
    event_ids = {"actualInteractionsPerCrossing", "averageInteractionsPerCrossing", "eventNumber", "mcChannelNumber", "runNumber"}
    mixins = {
        "globalTriggerEffSF": "Weight",  # events["NOSYS"].globalTriggerEffSF.emu
        "globalTriggerMatch": "Systematic",  # events["NOSYS"].globalTriggerMatch.emu
        **NtupleSchema.mixins
    }

events = NanoEventsFactory.from_root(
    {list(fileset[list(fileset.keys())[0]]["files"])[0]: "reco"},
    mode="virtual",
    schemaclass=TCPTSchema,
    entry_stop=1000,
    access_log=(access_log := []),
).events()

h = hist.new.Regular(30, 0, 300, label="leading electron $p_T$").StrCat([], name="variation", growth=True).Weight()

for variation in events.systematic_names:
    if variation != "NOSYS" and "EG_SCALE_ALL" not in variation:
        continue

    cut = events[variation]["pass"]["ejets"] == 1
    h.fill(events[variation][cut==1].el.pt[:, 0] / 1_000, variation=variation)

fig, ax = plt.subplots()
for variation in h.axes[1]:
    h[:, variation].plot(histtype="step", label=variation, ax=ax)
_ = ax.legend()

### Pre-processing

This step touches every file to extract metadata.

In [ ]:
run = processor.Runner(
    executor = processor.DaskExecutor(client=client, treereduction=4),
    # executor = processor.IterativeExecutor(),  # to run locally
    schema=TCPTSchema,
    savemetrics=True,
    chunksize=25_000,
    skipbadfiles=True,
    align_clusters=False,
    # maxchunks=1  # for debugging only
)


def extract_sumw(f):
    """read initial sum of weights, custom function to be injected during pre-processing"""
    matching_histograms = f.keys(filter_name="CutBookkeeper*NOSYS")
    if len(matching_histograms):
        sumw = float(f[matching_histograms[0]].values()[1])
    else:
        sumw = 0  # for data
    return {"sumw": sumw}


with performance_report(filename=f"{utils.get_output_path()}/preprocess.html"):
    # custom pre-processing: like coffea, but with more metadata
    t0 = time.time()
    preprocess_output = utils.custom_preprocess(fileset, client=client, chunksize=run.chunksize, custom_func=extract_sumw)
    t1 = time.time()


# calculate dataset-aggregated sumw (without chunk duplication) and update preprocessing metadata accordingly
sumw_dict = collections.defaultdict(dict)
for chunk in preprocess_output:
    sumw_dict[chunk.dataset][chunk.filename] = chunk.usermeta["sumw"]

for chunk in preprocess_output:
    # for data all sumw entries are 0, set the total to 1 manually
    chunk.usermeta.update({"sumw_dataset": sum(sumw_dict[chunk.dataset].values()) or 1})


# visualize task stream
ts = client.get_task_stream(start=f"{math.ceil(time.time()-t0)}s")
_ = utils.plot_taskstream(ts)

print(f"generated list of {len(preprocess_output)} work items in {t1-t0:.1f} sec:\n{preprocess_output[:3]}")

# write to disk
with open(f"{utils.get_output_path()}/preprocess_output.json", "w") as f:
    json.dump(utils.preprocess_to_json(preprocess_output), f)

### Processing

This either uses `coffea`'s `Runner` interface or a custom `dask.bag` method for more metadata tracking.
Configure this via `USE_CUSTOM_PROCESSING`.
The custom approach is needed for instantaneous data rate calculation in this notebook.

Track XCache egress: [link](https://grafana.mwt2.org/d/EKefjM-Sz/af-network-200gbps-challenge?orgId=1&from=now-1h&to=now&viewPanel=panel-205&refresh=5s)

In [ ]:
# wait for specific worker count
# import datetime
# nworker_target = 1
# while (nworkers := len(client.scheduler_info()['workers'])) < nworker_target:
#     print(datetime.datetime.now(), nworkers, end="\r")
#     time.sleep(1)
# print(datetime.datetime.now(), nworkers)

# configure whether to use coffea Runner interface or custom dask.bag-based version
USE_CUSTOM_PROCESSING = True


class Analysis(processor.ProcessorABC):
    def __init__(self):
        self.h = hist.new.Regular(20, 0, 1_000, label="$m_{jj}$ [GeV]").\
            StrCat([], name="category", growth=True).\
            StrCat([], name="variation", growth=True).\
            Weight()

    def get_weight(self, events, variation, cut, sumw):
        if events.metadata["dsid"] == "data":
            return 1.0

        # TODO: beamspot weight for mc20
        weight = events[variation][cut==1].weight.mc * events.metadata["weight_xs"] * events.metadata["lumi"] / sumw *\
                 events[variation][cut==1].weight.pileup * events[variation][cut==1].weight.jvt_effSF *\
                 events[variation][cut==1].weight.ftag_effSF_GN2v01_Continuous *\
                 events[variation][cut==1].weight.leptonSF_tight * events[variation][cut==1].globalTriggerEffSF.emu
        return weight

    def process(self, events):
        sumw = events.metadata["sumw_dataset"]  # TODO: systematics-dependent normalization
        for variation in events.systematic_names:
            if "EFF_" in variation:
                continue  # weight variations are handled in dedicated loop during nominal

            # only run over subset of systematics
            # if variation not in ["NOSYS"] + [name for name in events.systematic_names if "JET_JER_Effective" in name]:
            #     continue

            cut = events[variation]["pass"]["ejets"] == 1
            weight = self.get_weight(events, variation, cut, sumw)
            mjj = (events[variation][cut==1].jet[:, 0] + events[variation][cut==1].jet[:, 1]).mass
            self.h.fill(mjj / 1_000, category=events.metadata["category"], variation=variation, weight=weight)

            if variation == "NOSYS":
                # fill all weight variations during nominal
                for weight_sys_name in [var for var in events.systematic_names if "EFF_" in var]:
                    weight_var = self.get_weight(events, weight_sys_name, cut, sumw)
                    self.h.fill(mjj / 1_000, category=events.metadata["category"], variation=weight_sys_name, weight=weight_var)

        return {"hist": self.h, "meta": {}}

    def postprocess(self, accumulator):
        pass


# load pre-processing information from disk
with open(f"{utils.get_output_path()}/preprocess_output.json") as f:
    preprocess_output = utils.json_to_preprocess(json.load(f))

client.run_on_scheduler(utils.start_tracking)  # track worker count on scheduler
t0 = time.time()  # track walltime

if USE_CUSTOM_PROCESSING:
    # configure here whether to preload branches
    columns_to_preload = json.load(pathlib.Path("columns_to_preload.json").open())["all_systematics"]  # or [], ["JET_JER_Effective"]

    with performance_report(filename=f"{utils.get_output_path()}/process_custom.html"):
        out, report = utils.custom_process(preprocess_output, processor_class=Analysis, schema=run.schema, client=client, preload=columns_to_preload)

    print(f"preloaded columns: {len(columns_to_preload)}, {columns_to_preload[:4]} {"etc." if len(columns_to_preload) > 4 else ""}")
    print(f"preloaded but unused columns: {len([c for c in columns_to_preload if c not in report["columns"]])}")
    print(f"used but not preloaded columns: {len([c for c in report["columns"] if c not in columns_to_preload])}")

    # shortened version of report, dropping extra columns and per-chunk information
    display(
        dict((k, v) for k, v in report.items() if k not in ["columns", "chunk_info"]) | 
        {"columns": report["columns"][0:10] + ["..."]} | 
        {"chunk_info": list(report["chunk_info"].items())[:2] + ["..."]}
    )

else:
    # coffea Runner-based processing
    with performance_report(filename=f"{utils.get_output_path()}/process.html"):
        out, report = run(preprocess_output, processor_instance=Analysis())

    # shortened version of report, dropping extra columns
    display(dict((k, v) for k, v in report.items() if k != "columns") | ({"columns": report["columns"][0:10] + ["..."]}))

t1 = time.time()
worker_count_dict = client.run_on_scheduler(utils.stop_tracking)  # stop tracking, read out data, get average
nworker_avg = utils.get_avg_num_workers(worker_count_dict)

# histogram info
num_hists = 0
for cat in out["hist"].axes[1]:
    for sys in out["hist"][:, cat, :].axes[1]:
        if sum(out["hist"][:, cat, sys].values()) > 0.0:
            num_hists += 1

print(f"filled {num_hists} histograms, total size: {out["hist"].view(True).nbytes / 1_000 / 1_000:.2f} MB\n")

In [ ]:
print(f"walltime: {t1 - t0:.2f} sec ({(t1 - t0) / 60:.2f} min)")
print(f"average worker count: {nworker_avg:.1f}")
print(f"number of events processed: {report["entries"]:,}\n")

print(f"data read: {report["bytesread"] / 1000**3:.2f} GB in {report["chunks"]} chunks (average {report["bytesread"] / 1000**3 / report["chunks"]:.2f} GB per chunk)")
print(f"average total data rate: {report["bytesread"] / 1000**3 * 8 / (t1 - t0):.2f} Gbps")
print(f"fraction of input files read: {report["bytesread"] / 1000**3 / input_size_GB:.1%}")
print(f"number of branches read: {len(report["columns"])}\n")

print(f"worker-average event rate using \'processtime\': {report["entries"] / 1000 / report["processtime"]:.2f} kHz")
print(f"worker-average data rate using \'processtime\': {report["bytesread"] / 1000**3 * 8 / report["processtime"]:.2f} Gbps\n")

print(f"average event rate using walltime and time-averaged worker count: {report["entries"] / 1000 / (t1 - t0) / nworker_avg:.2f} kHz")
print(f"average data rate using walltime and time-averaged worker count: {report["bytesread"] / 1000**3 * 8 / (t1 - t0) / nworker_avg:.2f} Gbps\n")

print(f"fraction of time spent in processing: {report["processtime"] / ((t1 - t0) * nworker_avg):.1%}")
print(f"average process task length: {report["processtime"] / report["chunks"]:.1f} sec")

timestamps, datarates = utils.calculate_instantaneous_rates(t0, t1, report, num_samples=50)
_ = utils.plot_worker_count(worker_count_dict, timestamps, datarates)

# visualize task stream
ts = client.get_task_stream(start=f"{math.ceil(time.time()-t0)}s")
_ = utils.plot_taskstream(ts)

In [ ]:
mc_stack = []
labels = []
for cat in sorted(out["hist"].axes[1]):
    if cat in ["data", "ttbar_H7", "ttbar_hdamp", "ttbar_pthard", "Wt_DS", "Wt_H7", "Wt_pthard", "Hplus_cb"]:
        continue  # data drawn separately, skip MC modeling variations and signal

    mc_stack.append(out["hist"][:, cat, "NOSYS"])
    labels.append(cat)

try:
    data_hist = out["hist"][:, "data", "NOSYS"]
except KeyError:
    print("falling back to plotting first entry of categorical axes as \"data\"")
    data_hist = out["hist"][:, 0, 0]

fig, ax1, ax2 = mplhep.comp.data_model(
    data_hist=data_hist,
    stacked_components=mc_stack,
    stacked_labels=labels,
    # https://scikit-hep.org/mplhep/gallery/model_with_stacked_and_unstacked_histograms_components/
    # unstacked_components=[],
    # unstacked_labels=[],
    xlabel=out["hist"].axes[0].label,
    ylabel="Entries",
)

mplhep.atlas.label("Internal", ax=ax1, data=True, lumi=f"{utils.integrated_luminosity("", total=True) / 1000:.0f}", com="13/ \\ 13.6 \\ TeV")
mplhep.mpl_magic(ax=ax1)
ax2.set_ylim([0.5, 1.5])

# compare to e.g. https://atlas.web.cern.ch/Atlas/GROUPS/PHYSICS/PAPERS/HDBS-2020-11/fig_02a.png
fig.savefig(f"{utils.get_output_path()}/mjj.png")

In [ ]:
# save to disk
import uhi.io.json

with gzip.open(f"{utils.get_output_path()}/hist.json.gz", "w") as f:
    f.write(json.dumps(out["hist"], default=uhi.io.json.default).encode("utf-8"))

with gzip.open(f"{utils.get_output_path()}/hist.json.gz") as f:
    h = hist.Hist(json.loads(f.read(), object_hook=uhi.io.json.object_hook))

In [ ]:
evts_success = 0
evts_all = 0
num_failed = 0
for (fname, start, stop), (t0, t1, bytesread) in report["chunk_info"].items():
    if bytesread == 0:
        # print(fname, start, stop, t0, t1, bytesread)
        num_failed += 1
    else:
        evts_success += stop-start
    evts_all += stop-start
print(f"{num_failed} chunks failed")
print(f"{evts_success} events successful, {evts_success/evts_all:.2%} completion rate")